# Import and create a new SQLContext

In [90]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# Read the country CSV file into an RDD

In [91]:
country_lines = sc.textFile('file:///home/cloudera/Downloads/coursera-master/big-data-3/final-project/country-list.csv')
country_lines.take(7)

['Afghanistan, AFG',
 'Albania, ALB',
 'Algeria, ALG',
 'American Samoa, ASA',
 'Andorra, AND',
 'Angola, ANG',
 'Anguilla, AIA']

# Convert each line into a pair of words

In [92]:
country_lines.collect()
country_lines.map(lambda line : line.split(",")).take(7)

[['Afghanistan', ' AFG'],
 ['Albania', ' ALB'],
 ['Algeria', ' ALG'],
 ['American Samoa', ' ASA'],
 ['Andorra', ' AND'],
 ['Angola', ' ANG'],
 ['Anguilla', ' AIA']]

# Convert each pair of words into a tuple

In [93]:
country_tuples = country_lines.map(lambda line : (line.split(",")[0], 
                                                  line.split(",")[1]))
country_tuples.take(7)

[('Afghanistan', ' AFG'),
 ('Albania', ' ALB'),
 ('Algeria', ' ALG'),
 ('American Samoa', ' ASA'),
 ('Andorra', ' AND'),
 ('Angola', ' ANG'),
 ('Anguilla', ' AIA')]

# Create the DataFrame, look at schema and contents

In [94]:
countryDF = sqlContext.createDataFrame(country_tuples, ["country", "code"])
countryDF.printSchema()
countryDF.take(3)

root
 |-- country: string (nullable = true)
 |-- code: string (nullable = true)



[Row(country='Afghanistan', code=' AFG'),
 Row(country='Albania', code=' ALB'),
 Row(country='Algeria', code=' ALG')]

# Read tweets CSV file into RDD of lines

In [95]:
tweets = sc.textFile('file:///home/cloudera/Downloads/coursera-master/big-data-3/final-project/tweets.csv')
tweets.count()

13995

# Clean the data: Remove the empty tweets using filter() 

In [96]:
filtered_tweets = tweets.filter(lambda a : len(a) > 0)
filtered_tweets.count()

13391

# Perform WordCount on the cleaned tweet texts.

In [97]:
word_counts = filtered_tweets.flatMap(lambda line : line.split(" ")) \
    .map(lambda word : (word, 1)) \
    .reduceByKey(lambda a, b : (a + b))
word_counts.take(7)

[('', 3292),
 ('https://t.co/fQftAwGAad', 1),
 ('mobile', 1),
 ('#FridayNightTouchdown', 1),
 ('circle', 7),
 ('#thfc', 2),
 ('reinstated', 4)]

# Create the DataFrame of tweet word counts

In [98]:
tweetsDF = sqlContext.createDataFrame(word_counts, ["word", "count"])
tweetsDF.printSchema()
tweetsDF.take(7)

root
 |-- word: string (nullable = true)
 |-- count: long (nullable = true)



[Row(word='', count=3292),
 Row(word='https://t.co/fQftAwGAad', count=1),
 Row(word='mobile', count=1),
 Row(word='#FridayNightTouchdown', count=1),
 Row(word='circle', count=7),
 Row(word='#thfc', count=2),
 Row(word='reinstated', count=4)]

# Join the country and tweet DataFrames

In [99]:
joinedDF = countryDF.join(tweetsDF, countryDF.country == tweetsDF.word)
joinedDF.take(7)

[Row(country='Thailand', code=' THA', word='Thailand', count=1),
 Row(country='Iceland', code=' ISL', word='Iceland', count=2),
 Row(country='Mexico', code=' MEX', word='Mexico', count=1),
 Row(country='Wales', code=' WAL', word='Wales', count=19),
 Row(country='Denmark', code=' DEN', word='Denmark', count=1),
 Row(country='India', code=' IND', word='India', count=4),
 Row(country='Portugal', code=' POR', word='Portugal', count=8)]

# 1. Number of distinct countries mentioned

In [106]:
distinct_countries = joinedDF.select("country").distinct()
distinct_countries.count()

44

# 2. Number of countries mentioned in tweets.

In [105]:
from pyspark.sql.functions import sum

joinedDF.agg(sum("count")).first()

Row(sum(count)=397)

# 3. Top three countries and their counts.

In [104]:
from pyspark.sql.functions import desc
descSorted = joinedDF.sort(desc("count"))
descSorted.show(3)

+-------+----+-------+-----+
|country|code|   word|count|
+-------+----+-------+-----+
| Norway| NOR| Norway|   52|
|Nigeria| NGA|Nigeria|   49|
| France| FRA| France|   42|
+-------+----+-------+-----+
only showing top 3 rows



# 4. # times France was mentioned in a tweet

42

# 5. Most mentioned: Kenya, Wales, Netherlands

In [108]:
# Table 2: counts for Wales, Iceland, and Japan.
from pyspark.sql.functions import col

selected = joinedDF.where((col("country")=="Wales") | 
                          (col("country")=="Netherlands") |
                          (col("country")=="Kenya"))
selected.show()

+-----------+----+-----------+-----+
|    country|code|       word|count|
+-----------+----+-----------+-----+
|      Wales| WAL|      Wales|   19|
|Netherlands| NED|Netherlands|   13|
|      Kenya| KEN|      Kenya|    3|
+-----------+----+-----------+-----+



# 6. Average # times a country is mentioned

In [109]:
from pyspark.sql.functions import avg

joinedDF.agg(avg('count')).first()

Row(avg(count)=9.022727272727273)